In [1]:
import plotly.graph_objects as go
import plotly.io as pio
import pandas as pd

### Import & Filter Data

In [2]:
# Define Local Authority
LocalAuthority = '"Stirling"'
# Load Family Groups data file and extract the two relevant Family Groups for the selected Local Authority
FamilyGroups = pd.read_csv("Data Files//Family Groups.csv")
FamilyGroup_EnvEco = FamilyGroups.query(f'Local_Authority == {LocalAuthority} & Type == "Environmental, Culture & Leisure, Economic Development, Corporate and Property indicators"').iloc[0].Family_Group
FamilyGroup_ChildHouse = FamilyGroups.query(f'Local_Authority == {LocalAuthority} & Type == "Children, Social Work and Housing indicators"').iloc[0].Family_Group
# Load Family Group Averages data file, load and merge this with the indicator data and then use the family group types above to filter the averages for the appropriate Code and Family Group combinations
FamilyGroupData = pd.read_csv("Data Files//Family Averages.csv")
IndicatorInfo = pd.read_csv("Data Files//Indicator Information.csv")
FamilyGroupData = FamilyGroupData.merge(IndicatorInfo[['Code_Sortable','FamilyGrouping']], how = 'left', left_on = ['Code'], right_on = ['Code_Sortable'], suffixes=('_FGAverages', '_Info'))
FamilyGroupData_SelectedLA = FamilyGroupData.query(f'FamilyGrouping == "Environmental, Culture & Leisure, Economic Development, Corporate and Property indicators" & Family_Group == "{FamilyGroup_EnvEco}" | FamilyGrouping == "Children, Social Work and Housing indicators" & Family_Group == "{FamilyGroup_ChildHouse}"')
FamilyGroupData_SelectedLA = FamilyGroupData_SelectedLA[['Code','Period','FamilyAv_LA_Real','FamilyAv_LA_Cash','FamilyMed_LA_Real','FamilyMed_LA_Cash']]
# Load Indicator Data and filter it for the selected Local Authority
IndicatorData = pd.read_csv("Data Files//Indicator Data.csv")
IndicatorData_SelectedLA = IndicatorData.query(f'Local_Authority == {LocalAuthority}')[['Code', 'Period', 'Real_Value', 'Cash_Value', 'FamilyRank']]

# Print column names as reference
print("IndicatorData_SelectedLA")
print(list(IndicatorData_SelectedLA.columns.values))
print("FamilyGroupData_SelectedLA")
print(list(FamilyGroupData_SelectedLA.columns.values))
print("IndicatorInfo")
print(list(IndicatorInfo.columns.values))

IndicatorData_SelectedLA
['Code', 'Period', 'Real_Value', 'Cash_Value', 'FamilyRank']
FamilyGroupData_SelectedLA
['Code', 'Period', 'FamilyAv_LA_Real', 'FamilyAv_LA_Cash', 'FamilyMed_LA_Real', 'FamilyMed_LA_Cash']
IndicatorInfo
['Title', 'Code', 'Code_Sortable', 'ReportingPeriod', 'MeasureType', 'NumberFormat', 'YMin', 'YMax', 'ISCategory', 'Committee', 'FamilyGrouping', 'StirlingService', 'Ranking_Type', 'NumberFormat_NoText', 'Source', 'Numerator_Correct', 'Denominator_Correct', 'Numerator_Match', 'Denominator_Match', 'Numerator_Multipier', 'Denominator_Multiplier', 'Ranking_GoldilocksMidpoint', 'NumberFormat_Axis', 'Format_Python', 'FormatAxis_Python', 'AdditionalAxisDenominator_Python', 'FormatAxis_Plotly_Prefix', 'FormatAxis_Plotly_Suffix']


### Define Graph creation functions

In [3]:


# Define function to extract line series information filtered for the identified indicator code from the dataframes and combines into a single dataframe for the purpose of graphing.
def extractLineSeries (Code, LAData = IndicatorData_SelectedLA,FGData = FamilyGroupData_SelectedLA) :
    IndiData = LAData.query(f'Code == "{Code}"')[['Period','Real_Value','FamilyRank']]
    IndiData['FormattedFamilyRank'] = "#" + IndiData['FamilyRank'].map(int).map(str)
    FGData = FGData.query(f'Code == "{Code}"')[['Period','FamilyMed_LA_Real']]
    Data = FGData.merge(IndiData,how = 'left', on = ['Period'], suffixes = ('_FG','_Indi'))
    return Data

# Define function to create a standard styled line chart for a given set of data.
def createLineChart(Data,Format) :
    Chart = go.Figure()
    # Add local authority values as area line.
    Chart.add_trace(
        go.Scatter(
            x = Data['Period'],
            y = Data['Real_Value'],
            name = LocalAuthority ,
            mode ='lines',
            fill = 'tozeroy',
            line =
                dict(
                    color = 'purple',
                    width = 2,
                    shape = 'spline',
                    smoothing = 0.7 
                )
        )
    )
    # Add family group median values as line.
    Chart.add_trace(
        go.Scatter(
            x = Data['Period'],
            y = Data['FamilyMed_LA_Real'],
            name = 'Family Group Median',
            mode = 'lines',
            line = 
                dict(
                    color = 'red',
                    width = 2, 
                    shape = 'spline',
                    smoothing = 0.7 
                )
        )
    )
    # Format chart
    Chart.update_layout(
        xaxis_type ='category',
        plot_bgcolor =  'rgba(0, 0, 0, 0)',
        width = 700,
        height = 400,
        font = 
            dict(
                    family = "Segoe UI Semibold",
                    color = "black"
                ),
        margin = 
            dict(
                    b = 10,
                    l = 10,
                    r = 10,
                    t = 10,
                    pad = 10
                ),
        legend =
            dict(
                orientation = 'h',
                xanchor = 'left',
                x = 0,
                yanchor = 'top',
                y = 1.1,
                bgcolor = 'rgba(0, 0, 0, 0)',
                itemsizing = 'constant',
                font =
                    dict(
                        size = 12,
                    )
            ),
        yaxis = 
            dict(
                gridcolor = '#ededed',
                tickformat = Format,
                tickprefix = Prefix,
                ticksuffix = Suffix
            )
    )
    return Chart    

# Define a function to create a "Bump Chart" for use in visualising ranking change within family group.
def createBumpChart (Data) :
    Chart = go.Figure()
    Chart.add_trace(
        go.Scatter(
            x = Data['Period'],
            y = Data['FamilyRank'],
            text = Data['FormattedFamilyRank'],
            textposition = "middle center",
            textfont=
                dict(
                    family="Segoe UI Semibold",
                    size=12,
                    color="white"
                ),
            cliponaxis = False,
            name = f'{LocalAuthority} Family Group Rank',
            mode = 'lines+markers+text',
            line = 
                dict(
                    color = 'lightgray',
                    width = 10,
                    shape = 'spline',
                    smoothing = 0.7
                ),
            marker = 
                dict(
                    color = 'purple',
                    size = 25
                )
            )
    )
    Chart.update_yaxes(
        range = [8, 1],
        )
    Chart.update_layout(
        xaxis_type ='category',
        plot_bgcolor =  'rgba(0, 0, 0, 0)',
        width = 700,
        height = 400,
        font = 
            dict(
                    family = "Segoe UI Semibold",
                    color = "black"
                ),
        margin = 
            dict(
                    b = 10,
                    l = 10,
                    r = 10,
                    t = 10,
                    pad = 10
                )
    )
    return Chart 


## Create Graphs using Functions for each Indicator Code

In [4]:
for row in IndicatorInfo.itertuples() :
    # Filter main dataframes for particular indicator data and separate into arrays
    Code = row.Code_Sortable
    Format = row.FormatAxis_Python
    if pd.isnull(row.FormatAxis_Plotly_Prefix) :
        Prefix = ""
    else :
        Prefix = row.FormatAxis_Plotly_Prefix
    if pd.isnull(row.FormatAxis_Plotly_Suffix) :
        Suffix = ""
    else :
        Suffix = row.FormatAxis_Plotly_Suffix
    #Format = row.Format_Python
    Data = extractLineSeries(Code)
    # Create line charts and export these to png
    LineChart = createLineChart(Data,Format)
    LineChart.write_image(f'Graphs//{Code}_Line.png', scale = 23.622047244) # Aiming to output at 600 dpi
    # Create ranking charts and export these to png
    BumpChart = createBumpChart(Data)
    BumpChart.write_image(f'Graphs//{Code}_Bump.png', scale = 23.622047244) # Aiming to output at 600 dpi


#### Show Graph Examples

In [5]:
LineChart.show()
BumpChart.show()